In [25]:
from tqdm import tqdm
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
DB_HOST = 'localhost'
DB_NAME = 'us'
DB_USER = 'postgres'
DB_PASS = 'buddyrich134'
def get_db_connection():
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
    return conn


conn = get_db_connection()
cursor = conn.cursor()
import matplotlib.pyplot as plt
import warnings
import numpy as np

warnings.filterwarnings("ignore")
cursor.execute(f"SELECT distinct code from public.maincode where listed = 'us100';")
conn.commit()
list_ = [i[0] for i in cursor.fetchall()]
import pytz
tz = pytz.timezone('America/New_York')
da = datetime.now(tz).strftime("%Y-%m-%d")
print(da)

2024-09-11


In [26]:
# new --v 0.0.1 1min x all_pos/neg
def backtest_code(code):
    rolling = 5
    try:
        cursor.execute(f"SELECT da, code, cl from public.stock_price where code in ('{code}', 'QQQ') and da >= '{da} 09:30:00' order by da desc limit 12;")
        conn.commit()
        res = cursor.fetchall()
        df = pd.DataFrame(res)
        df.columns = ['da','code', 'cl']
        pivoted = df.pivot(columns='code', values='cl', index='da')
        pivoted.ffill()
        ret = pivoted.pct_change()
        ret['ret_diff'] = (ret[code] - ret['QQQ'])
        ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std() * 100
        ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean() * 100
        ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])/ret['ret_diff_std'])
        ret['code'] = ret[code]
        ret['code1'] = ret[code].shift(1)
        ret['code2'] = ret[code].shift(2)
        ret['code3'] = ret[code].shift(3)
        ret['code4'] = ret[code].shift(4)
        ret['all_positive'] = (ret['code'] > 0) & (ret['code1'] > 0)&(ret['code2'] > 0)&(ret['code3'] > 0)&(ret['code4'] > 0)
        ret['all_negative'] = (ret['code'] < 0) & (ret['code1'] < 0)&(ret['code2'] < 0)&(ret['code3'] < 0)&(ret['code4'] < 0)
        ret['all_pos'] = ret['all_positive'].apply(lambda x: 1 if x else 0)
        ret['all_neg'] = ret['all_negative'].apply(lambda x: 1 if x else 0)
        ret[code] = pivoted[code]
        ret['QQQ'] = pivoted['QQQ']
        ret['index'] = [i for i in range(len(ret))]

        ret['lag10'] = ret[code].shift(5) 
        ret['direction'] = ret.apply(lambda x: -1 if x[code] > x['lag10'] else 1, axis=1) # -1 if current larger than future but wrong
        
        ret['direction_icon'] = ret['direction'].apply(lambda x: '📈' if x == 1 else '📉')
        ret[f'{code}_30min_future'] = ret[code].shift(-60)
        ret['target_30mins'] = np.round((ret[f'{code}_30min_future'] - ret[code])/ret[code], 3) * ret['direction']
        ret['da'] = pivoted.index
        ret.set_index('index', inplace=True, drop=False)
        bm = 2

        ret = ret[(ret['stdize_ret_diff'] >= bm)]# & (((ret['direction'] == -1) & (ret['all_pos'] == 1)) | ((ret['direction'] == 1) & (ret['all_neg'] == 1)))]
        ret['code'] = code
        return ret.loc[:, ['direction_icon', 'all_neg', 'all_pos', 'stdize_ret_diff', f'target_30mins', 'da', 'code']]
    except:
        return None
    
    
dfs = []
for code in tqdm(list_):
    if code in ['CSGP']: continue
    df = backtest_code(code)
    if df is None: 
        continue
    dfs.append(df)
final = pd.concat(dfs)
das = final.sort_values(by='da', ascending=False)
# das = das[das['direction_icon'] == '📈']
das.sort_values(by=['direction_icon', 'da'])
# das.head(50)


100%|██████████| 98/98 [00:00<00:00, 109.21it/s]


code,direction_icon,all_neg,all_pos,stdize_ret_diff,target_30mins,da,code
index,,,,,,,
5,📈,1,0,2.914839,NaN,2024-09-11 14:50:00,PDD
5,📉,0,0,4.035797,NaN,2024-09-11 14:50:00,TMUS


In [27]:
# new --v 0.0.2 1min mean * std * 100000
def backtest_code(code):
    rolling = 5
    try:
        cursor.execute(f"SELECT da, code, cl from public.stock_price where code in ('{code}', 'QQQ') order by da desc limit 12;")
        conn.commit()
        res = cursor.fetchall()
        df = pd.DataFrame(res)
        df.columns = ['da','code', 'cl']
        pivoted = df.pivot(columns='code', values='cl', index='da')
        pivoted.ffill()
        ret = pivoted.pct_change()
        ret['ret_diff'] = (ret[code] - ret['QQQ'])
        ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std() * 100
        ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean() * 100
        ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])/ret['ret_diff_std'])

        ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std()
        ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean()
        ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])*ret['ret_diff_std'])*10000

        ret['code'] = ret[code]
        ret['code1'] = ret[code].shift(1)
        ret['code2'] = ret[code].shift(2)
        ret['code3'] = ret[code].shift(3)
        ret['code4'] = ret[code].shift(4)
        ret['all_positive'] = (ret['code'] > 0) & (ret['code1'] > 0)&(ret['code2'] > 0)&(ret['code3'] > 0)&(ret['code4'] > 0)
        ret['all_negative'] = (ret['code'] < 0) & (ret['code1'] < 0)&(ret['code2'] < 0)&(ret['code3'] < 0)&(ret['code4'] < 0)
        ret['all_pos'] = ret['all_positive'].apply(lambda x: 1 if x else 0)
        ret['all_neg'] = ret['all_negative'].apply(lambda x: 1 if x else 0)
        ret[code] = pivoted[code]
        ret['QQQ'] = pivoted['QQQ']
        ret['index'] = [i for i in range(len(ret))]

        ret['lag10'] = ret[code].shift(5) 
        ret['direction'] = ret.apply(lambda x: 1 if x[code] > x['lag10'] else -1, axis=1) # -1 if current larger than future but wrong
        
        ret['direction_icon'] = ret['direction'].apply(lambda x: '📈' if x == 1 else '📉')
        ret[f'{code}_30min_future'] = ret[code].shift(-60)
        ret['target_30mins'] = np.round((ret[f'{code}_30min_future'] - ret[code])/ret[code], 3) * ret['direction']
        ret['da'] = pivoted.index
        ret.set_index('index', inplace=True, drop=False)
        bm = 2

        ret = ret[(ret['stdize_ret_diff'] >= 2)]# & (((ret['direction'] == -1) & (ret['all_pos'] == 1)) | ((ret['direction'] == 1) & (ret['all_neg'] == 1)))]
        ret['code'] = code
        ret['hour'] = ret['da'].apply(lambda x: x.hour)
        ret['day'] = ret['da'].apply(lambda x: x.day)
        ret['min'] = ret['da'].apply(lambda x: int(x.minute))
        return ret.loc[:, ['direction_icon', 'all_neg', 'all_pos', 'stdize_ret_diff', f'target_30mins', 'da', 'code', 'hour', 'day', 'min']]
    except Exception  as e:
        return None
    
    
dfs = []
for code in tqdm(list_):
    df = backtest_code(code)
    if df is None: 
        continue
    dfs.append(df)
final = pd.concat(dfs)
das = final.sort_values(by='da', ascending=False)
# das.head(50)
das.sort_values(by=['direction_icon', 'da']).tail()


100%|██████████| 98/98 [00:00<00:00, 159.92it/s]


code,direction_icon,all_neg,all_pos,stdize_ret_diff,target_30mins,da,code,hour,day,min
index,,,,,,,,,,
